In [1]:
!pip install catboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 25.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [3]:
origin_train = pd.read_parquet(data_path + 'processed/train_all.parquet')
# origin_test = pd.read_parquet(data_path + 'processed/test_all.parquet')

In [4]:
segment = pd.read_csv(data_path + 'processed/segment.csv')

In [5]:
segment['mask'] = segment['Segment'] != 'E'
print(origin_train.index)
segment.reset_index(drop=True, inplace=True)
print(segment.index)

RangeIndex(start=0, stop=2400000, step=1)
RangeIndex(start=0, stop=2400000, step=1)


In [6]:
notESegment = segment.loc[segment['Segment'] != 'E',:'Segment']
notESegment

,ID,Segment
0,TRAIN_000000,D
2,TRAIN_000002,C
3,TRAIN_000003,D
8,TRAIN_000008,C
10,TRAIN_000010,D
...,...,...
2399979,TRAIN_399979,D
2399987,TRAIN_399987,C
2399993,TRAIN_399993,C
2399996,TRAIN_399996,D


In [7]:
train_abcd = origin_train.loc[segment['Segment'] != 'E',:]
del origin_train
del segment
gc.collect()

0

In [8]:
x = train_abcd.drop(['ID', '기준년월'], axis=1)
y = notESegment['Segment'].isin(['A','B']).astype(int)
del train_abcd
del notESegment
gc.collect()

0

In [9]:
print(x.shape)
y.value_counts()

(477948, 310)


,count
Segment,
0,476832
1,1116


In [10]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
x_scaled = scaler.fit_transform(x)

In [11]:
from sklearn.model_selection import train_test_split

x_train1, x_val, y_train1, y_val = train_test_split(x_scaled, y, test_size=0.2, random_state=42, stratify=y)
x_train, x_opt, y_train, y_opt = train_test_split(x_train1, y_train1, test_size=0.2, random_state=42, stratify=y_train1)

In [26]:
from imblearn.combine import SMOTETomek

sampler = SMOTETomek(sampling_strategy=0.15, random_state=42)
x_sample, y_sample = sampler.fit_resample(x_train, y_train)

In [27]:
y_sample.value_counts()

,count
Segment,
0,305139
1,45742


In [32]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

def cat_objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 6, 12),
        'class_weights': trial.suggest_float('class_weights', 1.0, 30.0),
        'border_count': trial.suggest_int('border_count', 64, 255),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    }

    clf = CatBoostClassifier(depth= params.get('depth'),
                             class_weights= [1.0, params.get('class_weights')],
                             border_count= params.get('border_count'),
                             l2_leaf_reg= params.get('l2_leaf_reg'),
                             learning_rate = params.get('learning_rate'),
                             iterations=1000, early_stopping_rounds=20,
                             task_type="GPU", devices='0',
                             verbose=False,
                             random_state=42)
    clf.fit(x_sample, y_sample, eval_set=(x_opt, y_opt))

    y_val_pred = clf.predict(x_val)
    f1_macro = f1_score(y_val, y_val_pred, average='macro')

    return f1_macro

In [33]:
study = optuna.create_study(direction='maximize')
study.optimize(cat_objective, n_trials=10)

print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-07-14 13:01:42,079] A new study created in memory with name: no-name-25413565-1bb3-4913-93a4-3eb2559c51dd
/tmp/ipython-input-32-1567812550.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2025-07-14 13:03:53,590] Trial 0 finished with value: 0.5675089205946567 and parameters: {'depth': 10, 'class_weights': 13.283305374624861, 'border_count': 145, 'l2_leaf_reg': 6, 'learning_rate': 6.878810209906319e-05}. Best is trial 0 with value: 0.5675089205946567.
/tmp/ipython-input-32-1567812550.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learnin

Best trial:
  Value: 0.6591242235181511
  Params: 
    depth: 12
    class_weights: 1.5431169466687404
    border_count: 207
    l2_leaf_reg: 5
    learning_rate: 0.0005914405521745386


In [34]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(depth=12,
                           border_count=207,
                           l2_leaf_reg=5,
                           learning_rate=5e-4,
                           early_stopping_rounds=20, iterations=1000,
                           class_weights=[1, 1.54],
                           task_type="GPU", devices='0',
                           random_state=42)
model.fit(x_sample, y_sample, eval_set=(x_opt, y_opt))

0:	learn: 0.6913806	test: 0.6911549	best: 0.6911549 (0)	total: 319ms	remaining: 5m 18s
1:	learn: 0.6895798	test: 0.6891370	best: 0.6891370 (1)	total: 524ms	remaining: 4m 21s
2:	learn: 0.6877099	test: 0.6870137	best: 0.6870137 (2)	total: 723ms	remaining: 4m
3:	learn: 0.6858835	test: 0.6849440	best: 0.6849440 (3)	total: 907ms	remaining: 3m 45s
4:	learn: 0.6840962	test: 0.6829500	best: 0.6829500 (4)	total: 1.11s	remaining: 3m 42s
5:	learn: 0.6822029	test: 0.6808040	best: 0.6808040 (5)	total: 1.3s	remaining: 3m 35s
6:	learn: 0.6803523	test: 0.6787270	best: 0.6787270 (6)	total: 1.5s	remaining: 3m 33s
7:	learn: 0.6785782	test: 0.6767359	best: 0.6767359 (7)	total: 1.7s	remaining: 3m 31s
8:	learn: 0.6767491	test: 0.6746725	best: 0.6746725 (8)	total: 1.9s	remaining: 3m 29s
9:	learn: 0.6750498	test: 0.6727638	best: 0.6727638 (9)	total: 2.1s	remaining: 3m 27s
10:	learn: 0.6732207	test: 0.6706956	best: 0.6706956 (10)	total: 2.29s	remaining: 3m 25s
11:	learn: 0.6714123	test: 0.6686617	best: 0.66866

In [35]:
from sklearn.metrics import f1_score

y_val_pred = model.predict(x_val)
f1_micro = f1_score(y_val, y_val_pred, average='micro')
print(f"F1 score (micro): {f1_micro:.4f}")

f1_macro = f1_score(y_val, y_val_pred, average='macro')
print(f"F1 score (macro): {f1_macro:.4f}")

F1 score (micro): 0.9907
F1 score (macro): 0.6559


In [36]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_val, y_val_pred)
print(conf_matrix)

[[94494   873]
 [   17   206]]


In [ ]:
from sklearn.metrics import roc_auc_score

y_val_pred_proba = model.predict_proba(x_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred_proba)

In [37]:
import pickle

with open(data_path + 'model/step2_OPTUNA.dat', 'wb') as fp:
    pickle.dump(model, fp)
    pickle.dump(scaler, fp)